In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, to_timestamp, col, when, sum, avg, count, split, explode, udf, collect_list, monotonically_increasing_id, row_number, size
from pyspark import Row
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.types import ArrayType, TimestampType
import datetime
from matplotlib import pyplot as plt

In [2]:
sc = SparkContext.getOrCreate()

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
df = spark.read \
          .option("sep", " ") \
          .option("inferSchema", "true") \
          .csv('data/2015_07_22_mktplace_shop_web_log_sample.log')

In [5]:
df.limit(10).toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13,_c14
0,2015-07-22T09:00:28.019143Z,marketpalce-shop,123.242.248.130:54635,10.0.6.158:80,0.000022,0.026109,0.000020,200,200,0,699,GET https://paytm.com:443/shop/authresponse?co...,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,ECDHE-RSA-AES128-GCM-SHA256,TLSv1.2
1,2015-07-22T09:00:27.894580Z,marketpalce-shop,203.91.211.44:51402,10.0.4.150:80,0.000024,0.153340,0.000026,200,200,0,1497,GET https://paytm.com:443/shop/wallet/txnhisto...,Mozilla/5.0 (Windows NT 6.1; rv:39.0) Gecko/20...,ECDHE-RSA-AES128-GCM-SHA256,TLSv1.2
2,2015-07-22T09:00:27.885745Z,marketpalce-shop,1.39.32.179:56419,10.0.4.244:80,0.000024,0.164958,0.000017,200,200,0,157,GET https://paytm.com:443/shop/wallet/txnhisto...,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,ECDHE-RSA-AES128-GCM-SHA256,TLSv1.2
3,2015-07-22T09:00:28.048369Z,marketpalce-shop,180.179.213.94:48725,10.0.6.108:80,0.000020,0.002333,0.000021,200,200,0,35734,GET https://paytm.com:443/shop/p/micromax-yu-y...,-,ECDHE-RSA-AES128-GCM-SHA256,TLSv1.2
4,2015-07-22T09:00:28.036251Z,marketpalce-shop,120.59.192.208:13527,10.0.4.217:80,0.000024,0.015091,0.000016,200,200,68,640,POST https://paytm.com:443/papi/v1/expresscart...,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,ECDHE-RSA-AES128-GCM-SHA256,TLSv1.2
5,2015-07-22T09:00:28.033793Z,marketpalce-shop,117.239.195.66:50524,10.0.6.195:80,0.000024,0.021570,0.000021,200,200,0,60,GET https://paytm.com:443/api/user/favourite?c...,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,ECDHE-RSA-AES128-GCM-SHA256,TLSv1.2
6,2015-07-22T09:00:28.055029Z,marketpalce-shop,101.60.186.26:33177,10.0.4.244:80,0.000020,0.001098,0.000022,200,200,0,1150,GET https://paytm.com:443/favicon.ico HTTP/1.1,Mozilla/5.0 (Windows NT 6.3; rv:27.0) Gecko/20...,ECDHE-RSA-AES128-GCM-SHA256,TLSv1.2
7,2015-07-22T09:00:28.050298Z,marketpalce-shop,59.183.41.47:62014,10.0.4.227:80,0.000021,0.008161,0.000021,200,200,0,72,GET https://paytm.com:443/papi/rr/products/693...,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,ECDHE-RSA-AES128-GCM-SHA256,TLSv1.2
8,2015-07-22T09:00:28.059081Z,marketpalce-shop,117.239.195.66:50538,10.0.4.227:80,0.000019,0.001035,0.000021,200,200,0,396,GET https://paytm.com:443/images/greyStar.png ...,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,ECDHE-RSA-AES128-GCM-SHA256,TLSv1.2
9,2015-07-22T09:00:28.054939Z,marketpalce-shop,183.83.237.83:49687,10.0.6.108:80,0.000023,0.008762,0.000021,200,200,0,214,GET https://paytm.com:443/shop/cart?channel=we...,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,ECDHE-RSA-AES128-GCM-SHA256,TLSv1.2


In [6]:
df.cache()

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: double, _c5: double, _c6: double, _c7: int, _c8: int, _c9: int, _c10: int, _c11: string, _c12: string, _c13: string, _c14: string]

In [7]:
df.dtypes

[('_c0', 'string'),
 ('_c1', 'string'),
 ('_c2', 'string'),
 ('_c3', 'string'),
 ('_c4', 'double'),
 ('_c5', 'double'),
 ('_c6', 'double'),
 ('_c7', 'int'),
 ('_c8', 'int'),
 ('_c9', 'int'),
 ('_c10', 'int'),
 ('_c11', 'string'),
 ('_c12', 'string'),
 ('_c13', 'string'),
 ('_c14', 'string')]

In [8]:
df.select('_c2').distinct().count()

404391

In [9]:
df.select('_c3').distinct().count()

25

In [10]:
df = df.withColumn('user_id', split('_c2', ':')[0])

In [11]:
df.select('user_id').show()

+---------------+
|        user_id|
+---------------+
|123.242.248.130|
|  203.91.211.44|
|    1.39.32.179|
| 180.179.213.94|
| 120.59.192.208|
| 117.239.195.66|
|  101.60.186.26|
|   59.183.41.47|
| 117.239.195.66|
|  183.83.237.83|
|  117.195.91.36|
|122.180.245.251|
| 117.198.215.20|
| 223.176.154.91|
|223.225.236.110|
| 117.241.97.140|
|117.205.247.140|
|   14.102.53.58|
|  203.200.99.67|
|107.167.109.204|
+---------------+
only showing top 20 rows



In [12]:
df.select('user_id').distinct().count()

90544

In [13]:
df = df.withColumn('occured_at', to_timestamp('_c0'))

In [14]:
df.limit(1).toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13,_c14,user_id,occured_at
0,2015-07-22T09:00:28.019143Z,marketpalce-shop,123.242.248.130:54635,10.0.6.158:80,0.000022,0.026109,0.00002,200,200,0,699,GET https://paytm.com:443/shop/authresponse?co...,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,ECDHE-RSA-AES128-GCM-SHA256,TLSv1.2,123.242.248.130,2015-07-22 06:00:28.019143


In [15]:
df.dtypes

[('_c0', 'string'),
 ('_c1', 'string'),
 ('_c2', 'string'),
 ('_c3', 'string'),
 ('_c4', 'double'),
 ('_c5', 'double'),
 ('_c6', 'double'),
 ('_c7', 'int'),
 ('_c8', 'int'),
 ('_c9', 'int'),
 ('_c10', 'int'),
 ('_c11', 'string'),
 ('_c12', 'string'),
 ('_c13', 'string'),
 ('_c14', 'string'),
 ('user_id', 'string'),
 ('occured_at', 'timestamp')]

## Processing & Analytical goals

## 1. Sessionize the web log by IP. Sessionize = aggregrate all page hits by visitor/IP during a session. https://en.wikipedia.org/wiki/Session_(web_analytics)

In [16]:
last_event = df.withColumn("last_event", lag('occured_at').over(Window.partitionBy('user_id').orderBy('occured_at')))

In [17]:
last_event.limit(6).toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13,_c14,user_id,occured_at,last_event
0,2015-07-22T16:14:36.308131Z,marketpalce-shop,1.186.143.37:11395,10.0.4.150:80,0.000023,0.004619,0.000020,200,200,0,13989,GET https://paytm.com:443/ HTTP/1.1,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:39.0) G...,ECDHE-RSA-AES128-GCM-SHA256,TLSv1.2,1.186.143.37,2015-07-22 13:14:36.308131,NaT
1,2015-07-22T16:14:44.846873Z,marketpalce-shop,1.186.143.37:11395,10.0.6.108:80,0.000025,0.008939,0.000021,200,200,0,215,GET https://paytm.com:443/shop/cart?channel=we...,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:39.0) G...,ECDHE-RSA-AES128-GCM-SHA256,TLSv1.2,1.186.143.37,2015-07-22 13:14:44.846873,2015-07-22 13:14:36.308131
2,2015-07-22T02:43:31.135201Z,marketpalce-shop,1.187.164.29:49414,10.0.4.244:80,0.000021,0.006038,0.000021,200,200,0,13980,GET https://paytm.com:443/shop?utm_source=Affi...,Mozilla/5.0 (Windows NT 6.1; rv:39.0) Gecko/20...,ECDHE-RSA-AES128-GCM-SHA256,TLSv1.2,1.187.164.29,2015-07-21 23:43:31.135201,NaT
3,2015-07-22T02:43:42.735304Z,marketpalce-shop,1.187.164.29:49414,10.0.6.158:80,0.000026,0.005846,0.000024,200,200,0,215,GET https://paytm.com:443/shop/cart?channel=we...,Mozilla/5.0 (Windows NT 6.1; rv:39.0) Gecko/20...,ECDHE-RSA-AES128-GCM-SHA256,TLSv1.2,1.187.164.29,2015-07-21 23:43:42.735304,2015-07-21 23:43:31.135201
4,2015-07-22T02:43:47.685225Z,marketpalce-shop,1.187.164.29:49414,10.0.6.199:80,0.000024,0.002448,0.000023,302,302,0,530,GET https://paytm.com:443/shop/login?isIframe=...,Mozilla/5.0 (Windows NT 6.1; rv:39.0) Gecko/20...,ECDHE-RSA-AES128-GCM-SHA256,TLSv1.2,1.187.164.29,2015-07-21 23:43:47.685225,2015-07-21 23:43:42.735304
5,2015-07-22T02:44:19.075268Z,marketpalce-shop,1.187.164.29:49414,10.0.4.225:80,0.000026,0.031224,0.000023,200,200,0,716,GET https://paytm.com:443/shop/authresponse?co...,Mozilla/5.0 (Windows NT 6.1; rv:39.0) Gecko/20...,ECDHE-RSA-AES128-GCM-SHA256,TLSv1.2,1.187.164.29,2015-07-21 23:44:19.075268,2015-07-21 23:43:47.685225


In [18]:
lag_in_day = last_event.withColumn('lag_in_day', (col('occured_at').astype("long") - col('last_event').astype("long"))/60)

In [19]:
lag_in_day.select('user_id', 'occured_at', 'last_event', 'lag_in_day').limit(20).toPandas()

,user_id,occured_at,last_event,lag_in_day
0,1.186.143.37,2015-07-22 13:14:36.308131,NaT,NaN
1,1.186.143.37,2015-07-22 13:14:44.846873,2015-07-22 13:14:36.308131,0.133333
2,1.187.164.29,2015-07-21 23:43:31.135201,NaT,NaN
3,1.187.164.29,2015-07-21 23:43:42.735304,2015-07-21 23:43:31.135201,0.183333
4,1.187.164.29,2015-07-21 23:43:47.685225,2015-07-21 23:43:42.735304,0.083333
5,1.187.164.29,2015-07-21 23:44:19.075268,2015-07-21 23:43:47.685225,0.533333
6,1.187.164.29,2015-07-21 23:44:20.084871,2015-07-21 23:44:19.075268,0.016667
7,1.187.164.29,2015-07-21 23:44:21.995557,2015-07-21 23:44:20.084871,0.016667
8,1.187.164.29,2015-07-21 23:44:23.254646,2015-07-21 23:44:21.995557,0.033333
9,1.187.164.29,2015-07-21 23:44:23.295011,2015-07-21 23:44:23.254646,0.000000


In [20]:
new_session = lag_in_day.withColumn('is_new_session', when( col('lag_in_day') > 15 , 1).otherwise(0))

In [21]:
new_session.select('user_id', 'occured_at', 'last_event', 'is_new_session').limit(20).toPandas()

,user_id,occured_at,last_event,is_new_session
0,1.186.143.37,2015-07-22 13:14:36.308131,NaT,0
1,1.186.143.37,2015-07-22 13:14:44.846873,2015-07-22 13:14:36.308131,0
2,1.187.164.29,2015-07-21 23:43:31.135201,NaT,0
3,1.187.164.29,2015-07-21 23:43:42.735304,2015-07-21 23:43:31.135201,0
4,1.187.164.29,2015-07-21 23:43:47.685225,2015-07-21 23:43:42.735304,0
5,1.187.164.29,2015-07-21 23:44:19.075268,2015-07-21 23:43:47.685225,0
6,1.187.164.29,2015-07-21 23:44:20.084871,2015-07-21 23:44:19.075268,0
7,1.187.164.29,2015-07-21 23:44:21.995557,2015-07-21 23:44:20.084871,0
8,1.187.164.29,2015-07-21 23:44:23.254646,2015-07-21 23:44:21.995557,0
9,1.187.164.29,2015-07-21 23:44:23.295011,2015-07-21 23:44:23.254646,0


In [22]:
new_session.select('is_new_session').describe().show()

+-------+--------------------+
|summary|      is_new_session|
+-------+--------------------+
|  count|             1158500|
|   mean|0.017514889943892966|
| stddev|  0.1311797752245047|
|    min|                   0|
|    max|                   1|
+-------+--------------------+



In [23]:
new_session.select(["occured_at", "user_id"]).where("is_new_session = 1").limit(10).toPandas()

,occured_at,user_id
0,2015-07-22 15:00:47.096589,1.23.208.26
1,2015-07-22 13:41:40.317662,1.39.32.249
2,2015-07-22 14:43:15.430441,1.39.33.77
3,2015-07-22 14:41:47.962868,1.39.63.5
4,2015-07-22 13:40:57.809077,1.39.80.59
5,2015-07-22 15:03:44.730731,1.39.96.177
6,2015-07-22 13:13:37.672328,103.15.63.34
7,2015-07-22 13:43:56.997234,103.15.63.34
8,2015-07-22 13:10:29.969176,103.16.71.9
9,2015-07-22 13:42:20.392273,103.16.71.9


In [24]:
new_session.select("user_id", "occured_at", "lag_in_day", "is_new_session").where("user_id = '1.23.208.26'").show()

+-----------+--------------------+--------------------+--------------+
|    user_id|          occured_at|          lag_in_day|is_new_session|
+-----------+--------------------+--------------------+--------------+
|1.23.208.26|2015-07-22 14:41:...|                null|             0|
|1.23.208.26|2015-07-22 14:41:...|0.016666666666666666|             0|
|1.23.208.26|2015-07-22 14:41:...|0.016666666666666666|             0|
|1.23.208.26|2015-07-22 14:41:...| 0.03333333333333333|             0|
|1.23.208.26|2015-07-22 14:41:...|                0.15|             0|
|1.23.208.26|2015-07-22 15:00:...|  18.883333333333333|             1|
+-----------+--------------------+--------------------+--------------+



In [25]:
user_session_id = new_session.withColumn("user_session_id", sum('is_new_session').over(Window.partitionBy('user_id').orderBy('occured_at')))

In [26]:
user_session_id.select('user_id', 'occured_at', 'last_event', 'user_session_id', "lag_in_day").show()

+------------+--------------------+--------------------+---------------+--------------------+
|     user_id|          occured_at|          last_event|user_session_id|          lag_in_day|
+------------+--------------------+--------------------+---------------+--------------------+
|1.186.143.37|2015-07-22 13:14:...|                null|              0|                null|
|1.186.143.37|2015-07-22 13:14:...|2015-07-22 13:14:...|              0| 0.13333333333333333|
|1.187.164.29|2015-07-21 23:43:...|                null|              0|                null|
|1.187.164.29|2015-07-21 23:43:...|2015-07-21 23:43:...|              0| 0.18333333333333332|
|1.187.164.29|2015-07-21 23:43:...|2015-07-21 23:43:...|              0| 0.08333333333333333|
|1.187.164.29|2015-07-21 23:44:...|2015-07-21 23:43:...|              0|  0.5333333333333333|
|1.187.164.29|2015-07-21 23:44:...|2015-07-21 23:44:...|              0|0.016666666666666666|
|1.187.164.29|2015-07-21 23:44:...|2015-07-21 23:44:...|    

In [27]:
user_session_id.select("user_id", "user_session_id", "lag_in_day").groupBy("user_id", "user_session_id").agg(count('*'), avg('lag_in_day')).show()

+------------+---------------+--------+--------------------+
|     user_id|user_session_id|count(1)|     avg(lag_in_day)|
+------------+---------------+--------+--------------------+
|1.186.143.37|              0|       2| 0.13333333333333333|
|1.187.164.29|              0|       9| 0.14375000000000002|
|  1.22.41.76|              0|       8| 0.14761904761904762|
| 1.23.208.26|              0|       5| 0.05416666666666667|
| 1.23.208.26|              1|       1|  18.883333333333333|
| 1.23.36.184|              0|       4| 0.08333333333333333|
|   1.38.19.8|              0|       2|                 1.0|
|  1.38.20.34|              0|      35| 0.09411764705882353|
|  1.39.13.13|              0|       2| 0.08333333333333333|
| 1.39.32.249|              0|       4| 0.31666666666666665|
| 1.39.32.249|              1|       2|  170.89166666666668|
|  1.39.32.59|              0|       1|                null|
| 1.39.33.153|              0|       8| 0.09761904761904762|
|  1.39.33.33|          

In [28]:
user_session_id.createOrReplaceTempView('user_session_id')

In [29]:
aggregated = spark.sql("""select user_id, user_session_id, count(*) as count, avg(lag_in_day) as avg_session_time from user_session_id group by user_id, user_session_id
""")

In [30]:
aggregated.show()

+------------+---------------+-----+--------------------+
|     user_id|user_session_id|count|    avg_session_time|
+------------+---------------+-----+--------------------+
|1.186.143.37|              0|    2| 0.13333333333333333|
|1.187.164.29|              0|    9| 0.14375000000000002|
|  1.22.41.76|              0|    8| 0.14761904761904762|
| 1.23.208.26|              0|    5| 0.05416666666666667|
| 1.23.208.26|              1|    1|  18.883333333333333|
| 1.23.36.184|              0|    4| 0.08333333333333333|
|   1.38.19.8|              0|    2|                 1.0|
|  1.38.20.34|              0|   35| 0.09411764705882353|
|  1.39.13.13|              0|    2| 0.08333333333333333|
| 1.39.32.249|              0|    4| 0.31666666666666665|
| 1.39.32.249|              1|    2|  170.89166666666668|
|  1.39.32.59|              0|    1|                null|
| 1.39.33.153|              0|    8| 0.09761904761904762|
|  1.39.33.33|              0|    2| 0.11666666666666667|
|  1.39.33.77|

## 2. Determine the average session time

In [31]:
aggregated.select('avg_session_time').agg(avg('avg_session_time')).show()

+---------------------+
|avg(avg_session_time)|
+---------------------+
|    14.60208471323375|
+---------------------+



## 3. Determine unique URL visits per session. To clarify, count a hit to a unique URL only once per session.

In [32]:
unique_ip = spark.sql(""" select user_id, user_session_id, count(*) as counts from (select distinct user_id, _c3 as visit, user_session_id from user_session_id) group by user_id, user_session_id
""")
unique_ip.show()

+------------+---------------+------+
|     user_id|user_session_id|counts|
+------------+---------------+------+
|1.186.143.37|              0|     2|
|1.187.164.29|              0|     7|
|  1.22.41.76|              0|     5|
| 1.23.208.26|              0|     4|
| 1.23.208.26|              1|     1|
| 1.23.36.184|              0|     4|
|   1.38.19.8|              0|     2|
|  1.38.20.34|              0|    12|
|  1.39.13.13|              0|     2|
| 1.39.32.249|              0|     4|
| 1.39.32.249|              1|     2|
|  1.39.32.59|              0|     1|
| 1.39.33.153|              0|     5|
|  1.39.33.33|              0|     2|
|  1.39.33.77|              0|     2|
|  1.39.33.77|              1|     3|
|   1.39.34.4|              0|     1|
|  1.39.40.43|              0|     5|
|  1.39.60.37|              0|    11|
|  1.39.61.53|              0|     7|
+------------+---------------+------+
only showing top 20 rows



## 4. Find the most engaged users, ie the IPs with the longest session times

In [33]:
spark.sql(""" select user_id
                   , user_session_id
                   , max_time 
              from (select user_id
                         , user_session_id
                         , max(lag_in_day) as max_time 
                    from user_session_id group by user_id
                                             , user_session_id) 
              order by max_time desc
""").show()

+---------------+---------------+------------------+
|        user_id|user_session_id|          max_time|
+---------------+---------------+------------------+
|   27.120.106.3|              1|1104.9833333333333|
|117.255.253.155|              1|            957.05|
|  103.24.125.26|              1| 954.7666666666667|
| 98.230.153.173|              1|            954.15|
| 150.228.40.140|              1| 953.5666666666667|
|   14.139.69.64|              1| 952.8666666666667|
|  66.249.82.186|              1| 952.1166666666667|
| 168.235.194.47|              1|             924.6|
|     1.39.35.89|              1| 922.3333333333334|
| 37.228.107.126|              1| 922.1166666666667|
|  163.47.14.170|              1|             922.0|
| 107.167.109.55|              1| 921.8833333333333|
|    1.39.63.157|              1|             921.7|
|  163.47.12.254|              1| 921.6666666666666|
|107.167.107.108|              1|            921.55|
|    1.39.12.226|              1| 921.38333333

# Saving Data

In [34]:
user_session_id.write.csv('data/user_session_id', header=True)

In [35]:
unique_ip.select('user_id', 'counts').write.csv('data/unique_ip', header=True)

In [36]:
spark.stop()